In [1]:
from pyspark.sql import SparkSession

spark =SparkSession.builder.appName("PySparkAssessment2").enableHiveSupport().getOrCreate()
spark

In [ ]:
import os

file_path = '/content/drive/MyDrive/PysparkDemo/large_employee_dataset.csv'
print(os.path.exists(file_path))

In [4]:
# 1. Ingestion & Exploration
# Read all 3 files (CSV + JSON) using PySpark.
# Show schemas and sample records.
# Count distinct departments.

from google.colab import drive
drive.mount('/content/drive',force_remount=True)


employeesdf =spark.read.csv('/content/drive/MyDrive/PysparkDemo/employees.csv',header=True,inferSchema=True)
attendancedf = spark.read.csv('/content/drive/MyDrive/PysparkDemo/attendance.csv',header=True,inferSchema=True)
bonusesdf = spark.read.option("multiline", True).json('/content/drive/MyDrive/PysparkDemo/bonuses.json')

employeesdf.printSchema()
employeesdf.show()

attendancedf.printSchema()
attendancedf.show()

bonusesdf.printSchema()
bonusesdf.show()

employeesdf.select("Department").distinct().show()

Mounted at /content/drive
root
 |-- EmpID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- JoinDate: date (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- ManagerID: integer (nullable = true)

+-----+------+-----------+----------+------+---------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|
+-----+------+-----------+----------+------+---------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|
|    3|Simran|Engineering|2022-07-10| 75000|        1|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|
|    5| Nisha|         HR|2023-01-05| 50000|        1|
+-----+------+-----------+----------+------+---------+

root
 |-- EmpID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Status: string (nullable = true)

+-----+----------+-------+
|EmpID|      Date| Status|
+-----+----------+-------+
|    1|2024-04-01|Present|
|    1|2024-0

In [6]:
# 2. DataFrame Operations
# Add a column TenureYears using datediff() and round() .
# Calculate TotalCompensation = Salary + Bonus .
# Filter employees with more than 2 years in the company.
# Show employees who report to a manager ( ManagerID is not null ).

from pyspark.sql.functions import to_date,datediff,col,round,current_date

employeesdf = employeesdf.withColumn("JoinDate", to_date("JoinDate"))
employeesdf = employeesdf.withColumn("TenureYears", round(datediff(current_date(), col("JoinDate")) / 365, 2))

emp_with_bonus = employeesdf.join(bonusesdf, "EmpID")
emp_with_bonus = emp_with_bonus.withColumn("TotalCompensation", col("Salary") + col("Bonus"))


emp_with_bonus.filter(col("TenureYears") > 2).show()


employeesdf.filter(col("ManagerID").isNotNull()).show()



+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|       4.11| 5000|2023|            60000|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|       5.24| 7000|2023|            87000|
|    3|Simran|Engineering|2022-07-10| 75000|        1|       2.92| 6500|2023|            81500|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|       5.56| 6000|2023|            66000|
|    5| Nisha|         HR|2023-01-05| 50000|        1|       2.43| 4000|2023|            54000|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+

+-----+------+-----------+----------+------+---------+-----------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|
+

In [9]:
# 3. Aggregation
# Average salary per department.
# Number of employees under each manager.
# Count of absences per employee.

employeesdf.groupBy("Department").agg({"Salary": "avg"}).show()
employeesdf.groupBy("ManagerID").agg({"EmpID": "count"}).show()
attendancedf.filter(col("Status") == "Absent").groupBy("EmpID").count().withColumnRenamed("count", "AbsenceCount").show()

+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|Engineering|    77500.0|
|         HR|    52500.0|
|  Marketing|    60000.0|
+-----------+-----------+

+---------+------------+
|ManagerID|count(EmpID)|
+---------+------------+
|     NULL|           1|
|        1|           4|
+---------+------------+

+-----+------------+
|EmpID|AbsenceCount|
+-----+------------+
|    4|           2|
|    2|           1|
+-----+------------+



In [10]:
# 4. Joins
# Join employees and attendance → Get attendance % (Present days / Total days).
# Join employees and bonuses → Show top 3 employees by TotalCompensation.
# Multi-level join: employees + bonuses + attendance .

from pyspark.sql.functions import count, sum

attendance_grouped = attendancedf.groupBy("EmpID").pivot("Status").count().fillna(0)
attendance_grouped = attendance_grouped.withColumn("AttendancePercent",round(col("Present") / (col("Present") + col("Absent")) * 100, 2))

employeesdf.join(attendance_grouped, "EmpID", "left").select("EmpID", "Name", "AttendancePercent").show()

emp_with_bonus.orderBy(col("TotalCompensation").desc()).select("EmpID", "Name", "TotalCompensation").show(3)

joined_df = employeesdf.join(bonusesdf, "EmpID").join(attendance_grouped, "EmpID")
joined_df.show()


+-----+------+-----------------+
|EmpID|  Name|AttendancePercent|
+-----+------+-----------------+
|    1| Anita|            100.0|
|    2|   Raj|             50.0|
|    3|Simran|            100.0|
|    4| Aamir|              0.0|
|    5| Nisha|            100.0|
+-----+------+-----------------+

+-----+------+-----------------+
|EmpID|  Name|TotalCompensation|
+-----+------+-----------------+
|    2|   Raj|            87000|
|    3|Simran|            81500|
|    4| Aamir|            66000|
+-----+------+-----------------+
only showing top 3 rows

+-----+------+-----------+----------+------+---------+-----------+-----+----+------+-------+-----------------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|Absent|Present|AttendancePercent|
+-----+------+-----------+----------+------+---------+-----------+-----+----+------+-------+-----------------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|       4.11| 5000|2023|     0|      2|            100.0|
| 

In [11]:
# 5. String & Date Functions
# Extract year and month from JoinDate .
# Mask employee names using regex.
# Use substring() to create EmpCode like "EMP001".

from pyspark.sql.functions import year, month, regexp_replace, lpad, concat, lit

employeesdf = employeesdf.withColumn("JoinYear", year("JoinDate")) \
                     .withColumn("JoinMonth", month("JoinDate"))


employeesdf = employeesdf.withColumn("MaskedName", regexp_replace("Name", r"(?<=.).(?=.)", "*"))


employeesdf = employeesdf.withColumn("EmpCode", concat(lit("EMP"), lpad(col("EmpID").cast("string"), 3, "0")))

employeesdf.select("EmpID", "Name", "MaskedName", "EmpCode", "JoinYear", "JoinMonth").show()


+-----+------+----------+-------+--------+---------+
|EmpID|  Name|MaskedName|EmpCode|JoinYear|JoinMonth|
+-----+------+----------+-------+--------+---------+
|    1| Anita|     A***a| EMP001|    2021|        5|
|    2|   Raj|       R*j| EMP002|    2020|        3|
|    3|Simran|    S****n| EMP003|    2022|        7|
|    4| Aamir|     A***r| EMP004|    2019|       11|
|    5| Nisha|     N***a| EMP005|    2023|        1|
+-----+------+----------+-------+--------+---------+



In [12]:
# 6. Conditional & Null Handling
# Use when/otherwise to label performance:
# “High” if Bonus > 6000
# “Medium” if 4000–6000
# “Low” otherwise
# Handle missing ManagerID using fillna("No Manager") .
from pyspark.sql.functions import when

emp_with_bonus = emp_with_bonus.withColumn("Performance",when(col("Bonus") > 6000, "High").when((col("Bonus") >= 4000) & (col("Bonus") <= 6000), "Medium").otherwise("Low"))

employees_filled = employeesdf.fillna({"ManagerID": "No Manager"})

emp_with_bonus.select("EmpID", "Name", "Bonus", "Performance").show()
employees_filled.select("EmpID", "Name", "ManagerID").show()


+-----+------+-----+-----------+
|EmpID|  Name|Bonus|Performance|
+-----+------+-----+-----------+
|    1| Anita| 5000|     Medium|
|    2|   Raj| 7000|       High|
|    3|Simran| 6500|       High|
|    4| Aamir| 6000|     Medium|
|    5| Nisha| 4000|     Medium|
+-----+------+-----+-----------+

+-----+------+---------+
|EmpID|  Name|ManagerID|
+-----+------+---------+
|    1| Anita|     NULL|
|    2|   Raj|        1|
|    3|Simran|        1|
|    4| Aamir|        1|
|    5| Nisha|        1|
+-----+------+---------+



In [14]:
# 7. Spark SQL
# Create and use database hr .
# Save all DataFrames as tables: employees , attendance , bonuses .
# Write SQL queries:
# Top paid employee in each department.
# Attendance rate by department.
# Employees joined after 2021 with salary > 70,000.

employeesdf.createOrReplaceTempView("employees")
attendancedf.createOrReplaceTempView("attendance")
bonusesdf.createOrReplaceTempView("bonuses")

spark.sql("""
    SELECT Department, Name, Salary
    FROM (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY Department ORDER BY Salary DESC) AS rn
        FROM employees
    ) WHERE rn = 1
""").show()

attendancedf.createOrReplaceTempView("attendance")
spark.sql("""
    SELECT e.Department,
           ROUND(SUM(CASE WHEN a.Status = 'Present' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS AttendanceRate
    FROM employees e
    JOIN attendance a ON e.EmpID = a.EmpID
    GROUP BY e.Department
""").show()

spark.sql("""
    SELECT *
    FROM employees
    WHERE JoinDate > '2021-12-31' AND Salary > 70000
""").show()



+-----------+-----+------+
| Department| Name|Salary|
+-----------+-----+------+
|Engineering|  Raj| 80000|
|         HR|Anita| 55000|
|  Marketing|Aamir| 60000|
+-----------+-----+------+

+-----------+--------------+
| Department|AttendanceRate|
+-----------+--------------+
|Engineering|         75.00|
|         HR|        100.00|
|  Marketing|          0.00|
+-----------+--------------+

+-----+------+-----------+----------+------+---------+-----------+--------+---------+----------+-------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|JoinYear|JoinMonth|MaskedName|EmpCode|
+-----+------+-----------+----------+------+---------+-----------+--------+---------+----------+-------+
|    3|Simran|Engineering|2022-07-10| 75000|        1|       2.92|    2022|        7|    S****n| EMP003|
+-----+------+-----------+----------+------+---------+-----------+--------+---------+----------+-------+



In [17]:
# 8. Advanced (Optional)
# Use a UDF to classify department as "Tech" vs "Non-Tech".
# Create a view emp_attendance_summary .
# Save it as Parquet partitioned by Department .

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def classify_department(dept):
    return "Tech" if dept in ["Engineering"] else "Non-Tech"

dept_udf = udf(classify_department, StringType())
employeesdf = employeesdf.withColumn("DeptType", dept_udf("Department"))

emp_attendance_summary = joined_df.select("EmpID", "Name", "Department", "Present", "Absent", "AttendancePercent")
emp_attendance_summary.createOrReplaceTempView("emp_attendance_summary")

emp_attendance_summary.write.mode("overwrite").partitionBy("Department").parquet("/tmp/emp_attendance_summary")
emp_attendance_summary.show()



+-----+------+-----------+-------+------+-----------------+
|EmpID|  Name| Department|Present|Absent|AttendancePercent|
+-----+------+-----------+-------+------+-----------------+
|    1| Anita|         HR|      2|     0|            100.0|
|    2|   Raj|Engineering|      1|     1|             50.0|
|    3|Simran|Engineering|      2|     0|            100.0|
|    4| Aamir|  Marketing|      0|     2|              0.0|
|    5| Nisha|         HR|      2|     0|            100.0|
+-----+------+-----------+-------+------+-----------------+

